- [Done] Prepare .containerfile and .containerignore
- [Done] Build container
- [Done] Put files into the container
- [Done] Run container
- [Done] Get files from the container
- [Done] Kill the container

In [ ]:
# Open the automation_folder
from pathlib import Path
examples_folder = Path('~/Projects/crosscompute-examples').expanduser()
examples_folder

In [ ]:
automation_folder = examples_folder / 'tools' / 'add-numbers'
automation_folder

In [ ]:
ls $automation_folder -a

In [ ]:
from crosscompute.routines.configuration import load_configuration
configuration = load_configuration(automation_folder / 'automate.yml')
configuration

In [ ]:
from crosscompute.constants import MODE_NAMES
mode_folder_by_name = {_ + '_folder': 'runs/next/' + _ for _ in MODE_NAMES}
mode_folder_by_name

In [ ]:
script_definitions = configuration.script_definitions
command_strings = [_.get_command_string() for _ in script_definitions]
command_strings

In [ ]:
import shlex

In [ ]:
command_texts = [_.format(**mode_folder_by_name) for _ in command_strings]
command_texts

In [ ]:
PIPE_TEXT = ' 1>runs/debug/stdout.txt 2>runs/debug/stderr.txt'
command_texts = [_ + PIPE_TEXT for _ in command_texts]

In [ ]:
command_lists = [shlex.split(_) for _ in command_texts]
command_lists

In [ ]:
bash_script_text = '\n'.join(command_texts)
bash_script_text

In [ ]:
base_image_tag_name = configuration['environment']['container']['image']

In [ ]:
# Prepare .containerignore
CONTAINER_IGNORE_TEXT = '''\
.containerfile
.containerignore
.gitignore
.ipynb_checkpoints
batches/
runs/
tests/'''
container_ignore_name = '.containerignore'
(automation_folder / container_ignore_name).open('wt').write(CONTAINER_IGNORE_TEXT)

In [ ]:
# Prepare .containerfile
from jinja2 import Template
CONTAINER_FILE_TEXT = Template('''\
FROM {{ base_image_tag_name }}
RUN useradd user
USER user
WORKDIR /home/user
COPY --chown=user:user . .
RUN mkdir runs/next/input runs/next/log runs/next/debug runs/next/output -p
CMD ["sleep", "infinity"]''')
container_file_text = CONTAINER_FILE_TEXT.render(
    base_image_tag_name=base_image_tag_name)
container_file_name = '.containerfile'
(automation_folder / container_file_name).open('wt').write(container_file_text)
print(container_file_text)

In [ ]:
automation_definition = configuration.automation_definitions[0]

In [ ]:
automation_name = automation_definition.name
automation_slug = automation_definition.slug
automation_version = automation_definition.version

In [ ]:
image_tag_name = f'{automation_slug}:{automation_version}'
image_tag_name

In [ ]:
import subprocess

In [ ]:
subprocess.run(['podman', 'build', '-t', image_tag_name, '-f', container_file_name], cwd=automation_folder)

In [ ]:
subprocess.run(['podman', 'images'])

In [ ]:
process = subprocess.run(['podman', 'run', '-d', image_tag_name], capture_output=True)

In [ ]:
container_id = process.stdout.decode().strip()
container_id

In [ ]:
subprocess.run(['podman', 'ps'])

In [ ]:
subprocess.run(['podman', 'cp', 'runs/2-3/input', container_id + ':runs/next/'], cwd=automation_folder)

In [ ]:
subprocess.run(['podman', 'exec', container_id, 'python', 'run.py', 'runs/next/input', 'runs/next/output'], cwd=automation_folder)

In [ ]:
subprocess.run(['podman', 'cp', container_id + ':runs/next/.', 'runs/2-3'], cwd=automation_folder)

In [ ]:
subprocess.run(['podman', 'kill', container_id])